In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# voc = [0,2,3,4,8,9,10] + list(range(32,51))
voc = [0,2,3,4,8,9,10] + list(range(32,51))

pm25 = list(range(11,30))

no = [6,7]

rest = [1,5,30,31]

In [3]:
P_emis_51 = np.load('/AS_data/Emis_with_foreign_npy/EM_2015_01_744_51_182_232.npy')
print(P_emis_51.shape)

P_emis_7 = np.load('/AS_data/zeyuan_folder/concat_data/rest_EM_2015_01_744_07_182_232.npy')
print(P_emis_7.shape)

N_emis_7 = np.load('/AS_data/finetune_input_prelu_VOCsame_PM25_O3_NO2_SO2/rest_4month_SULF_CO_7dim_01_finetune_input.npy')
print(N_emis_7.shape)

(744, 51, 182, 232)
(744, 7, 182, 232)
(744, 7, 182, 232)


In [4]:
N_emis_51 = np.zeros_like(P_emis_51)

for idxes51,idx7 in [[voc,4],[pm25,5],[no,6]]:
    print(idxes51,idx7)
    N_emis_51[:,idxes51] = P_emis_51[:,idxes51] * (N_emis_7[:,idx7:idx7+1]+1e-5) / (P_emis_7[:,idx7:idx7+1]+1e-5)
    

N_emis_51[:,rest] = P_emis_51[:,rest] * (N_emis_7[:,np.array([0,1,2,3])]+1e-5) / (P_emis_7[:,np.array([0,1,2,3])]+1e-5)

[0, 2, 3, 4, 8, 9, 10, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50] 4
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] 5
[6, 7] 6


In [ ]:

#
data = []
# 1,32   91,122  182,213    274,305   
for i in range(274,305):
    data.append(nc.Dataset('/scratch/AS_data/Emission_copy/EM_2015{}'.format(i), 'a'))
print(len(data))
print(type(data[0]))


var_keys = [name for name in data[0].variables]
var_keys = var_keys[1:]
print(type(var_keys))
print(var_keys)
print(len(var_keys))

order = ['ALD2', 'CO', 'ETH', 'FORM', 'ISOP', 'NH3', 'NO', 'NO2', 'UNR', 'OLE',\
 'PAR', 'PEC', 'PMC', 'PMOTHR', 'PNO3', 'POC', 'PSO4', 'PCL', 'PNH4', 'PNA',\
 'PMG', 'PK', 'PCA', 'PNCOM', 'PFE', 'PAL', 'PSI', 'PTI', 'PMN', 'PH2O',\
 'SO2', 'SULF', 'TERP', 'TOL', 'XYL', 'MEOH', 'ETOH', 'ETHA', 'ALDX', 'IOLE',\
 'CH4', 'AACD', 'NAPH', 'NR', 'SOAALK', 'XYLMN', 'PRPA', 'BENZ', 'ETHY', 'ACET',\
 'KET']

if var_keys!=order:
    assert False

In [ ]:
for i in range(len(data)): 
    variable = data[i].variables
    for em in range(51):
        em_data = deepcopy(variable[var_keys[em]][:-1])
        sum_em = np.sum(em_data, axis = 1, keepdims = True)
        
        final_em = N_emis_51[i*24:i*24+24,:,em,:,:]
        
        finetune_em = final_em * em_data  / sum_em
        variable[var_keys[em]][:-1] = finetune_em
    print('Process {}/{}'.format(i,len(data)))

(744, 51, 182, 232)
(744, 7, 182, 232)


In [22]:
#from zeyuan
org_data = P_emis_51
con_data = P_emis_7
fin_data = N_emis_7


from copy import deepcopy
final_data = deepcopy(org_data)
delta_data = fin_data - con_data
print(final_data.shape)
print(delta_data.shape)

print('Updating VOC')
delta_voc = delta_data[:,4,:,:]
mean_voc = con_data[:,4,:,:]
for i in voc:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)
print('Updating PM25')
delta_voc = delta_data[:,5,:,:]
mean_voc = con_data[:,5,:,:]
for i in pm25:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)
print('Updating NO')
delta_voc = delta_data[:,6,:,:]
mean_voc = con_data[:,6,:,:]
for i in no:
    final_data[:,i,:,:] += (org_data[:,i,:,:] + 1e-5) * delta_voc / (mean_voc + 1e-5)
for i in range(4):
    final_data[:,rest[i],:,:] += delta_data[:,i,:,:]
    
    
with open('/AS_data/zeyuan_folder/reload_data/adjust_new_7EM_07_744_51_182_232.npy','wb') as f:
    np.save(f,final_data)

Updating VOC
Updating PM25
Updating NO
